### about the data

This is a transactional data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

the data can be download at: https://archive.ics.uci.edu/dataset/352/online+retail

##### description

InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 

StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.

Description: Product (item) name. Nominal.

Quantity: The quantities of each product (item) per transaction. Numeric.	

InvoiceDate: Invoice Date and time. Numeric, the day and time when each transaction was generated.

UnitPrice: Unit price. Numeric, Product price per unit in sterling.

CustomerID: Customer number. Nominal, a 5-digit integral 
number uniquely assigned to each customer.

Country: Country name. Nominal, the name of the country where each customer resides. 

In [40]:
# Requirements - write to requirements.txt
req = '''
streamlit==1.26.0
pandas==2.1.2
sqlalchemy==2.1.0
plotly==5.16.1
python-dateutil==2.8.2
numpy==1.26.2
scipy==1.11.3
matplotlib==3.8.1
seaborn==0.12.2
ipython
jupyter
'''
with open('../requirements.txt','w') as f:
    f.write(req)
print('wrote requirements.txt')

wrote requirements.txt


In [41]:
readme = '''# Online Retail — SQL + Streamlit

This repo contains an end-to-end demo project: ETL from the Online Retail CSV into SQLite, SQL analysis, and Streamlit dashboard skeleton workflow.

#### Files:
- db.py: SQLite helpers
- etl.py: cleans CSV and populates normalized tables
- queries.sql: useful SQL snippets
- app.py: Streamlit dashboard skeleton (KPIs, cohort, top products, RFM)
- requirements.txt

#### About the ddata

This is a transactional data set which contains all the transactions occurring between 01/12/2010 
and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique 
all-occasion gifts. Many customers of the company are wholesalers.

the data can be download at: https://archive.ics.uci.edu/dataset/352/online+retail

InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 

StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.

Description: Product (item) name. Nominal.

Quantity: The quantities of each product (item) per transaction. Numeric.	

InvoiceDate: Invoice Date and time. Numeric, the day and time when each transaction was generated.

UnitPrice: Unit price. Numeric, Product price per unit in sterling.

CustomerID: Customer number. Nominal, a 5-digit integral 
number uniquely assigned to each customer.

Country: Country name. Nominal, the name of the country where each customer resides. 

#### Setup (Ubuntu):
1. install requirements
   pip install -r requirements.txt
2. run ETL
   python etl.py /path/to/OnlineRetail.csv
3. run Streamlit app
   streamlit run app.py
'''
with open('../README.md','w') as f:
    f.write(readme)
print('wrote README.md')

gitignore = '''
venv/
data/online_retail.db
__pycache__/
.ipynb_checkpoints/
'''
with open('../.gitignore','w') as f:
    f.write(gitignore)
print('wrote .gitignore')

wrote README.md
wrote .gitignore


In [5]:
##creating database

In [44]:
from sqlalchemy import create_engine, text
import sys
from pathlib import Path
import pandas as pd
#from db import ENGINE

p = Path('../data/Online-Retail.xlsx')

DB_PATH = Path(p).parent / 'online_retail.db'
DB_PATH.parent.mkdir(parents=True, exist_ok=True)
ENGINE = create_engine(f"sqlite:///{DB_PATH}", connect_args={'check_same_thread': False})

def run_sql(query: str, params: dict = None) -> pd.DataFrame:
    with ENGINE.connect() as conn:
        return pd.read_sql_query(text(query), conn, params=params)

def execute_script(script: str):
    with ENGINE.begin() as conn:
        conn.execute(text(script))

In [47]:
db_py = r"""
from sqlalchemy import create_engine, text
import pandas as pd
from pathlib import Path

DB_PATH = Path(__file__).parent / 'data' / 'online_retail.db'
DB_PATH.parent.mkdir(parents=True, exist_ok=True)
ENGINE = create_engine(f"sqlite:///{DB_PATH}", connect_args={"check_same_thread": False})

def run_sql(query: str, params: dict = None) -> pd.DataFrame:
    with ENGINE.connect() as conn:
        return pd.read_sql_query(text(query), conn, params=params)

def execute_script(script: str):
    with ENGINE.begin() as conn:
        conn.execute(text(script))

if __name__ == '__main__':
    print('DB helpers ready. DB file:', DB_PATH)
"""
with open('../db.py','w') as f:
    f.write(db_py)
print('wrote db.py')

wrote db.py


### etl

#!pip3 install -U openpyxl

In [46]:
df = pd.read_excel(p)
df.head()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [4]:
df = df.dropna(subset=['CustomerID', 'InvoiceNo', 'StockCode', 'InvoiceDate'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [5]:
#getting rid of cancelled transactions
df[df['InvoiceNo'].astype(str).str.startswith('C')]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [6]:
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [7]:
df['InvoiceDate']

0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: InvoiceDate, Length: 397924, dtype: datetime64[ns]

In [8]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce')

In [9]:
df = df[df['InvoiceDate'].notna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [10]:
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(0).astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [11]:
df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce').fillna(0.0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [12]:
df = df[df['Quantity'] >0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [13]:
df['CustomeID'] = df['CustomerID'].astype(int)
df['StockCode'] = df['StockCode'].astype(str)
df['Description'] = df['Description'].fillna('').str.strip()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
 8   CustomeID    397924 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 30.4+ MB


In [14]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CustomeID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,17850


In [48]:
with ENGINE.begin() as conn:
    conn.execute(text('DROP TABLE IF EXISTS staging;'))
df.to_sql('staging', con=ENGINE, if_exists='replace', index=False)

541909

In [24]:
with ENGINE.begin() as conn:
    conn.execute(text("""
    CREATE TABLE IF NOT EXISTS customers (
        customer_id INTEGER PRIMARY KEY,
        country TEXT
    );
    """))
    conn.execute(text("""
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY AUTOINCREMENT,
        stock_code TEXT UNIQUE,
        description TEXT
    );
    """))
    conn.execute(text("""
    CREATE TABLE IF NOT EXISTS orders (
        order_id INTEGER PRIMARY KEY,
        invoice_no TEXT,
        invoice_date TIMESTAMP,
        customer_id INTEGER,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );
    """))
    conn.execute(text("""
    CREATE TABLE IF NOT EXISTS order_items (
        order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
        order_id INTEGER,
        product_id INTEGER,
        quantity INTEGER,
        unit_price REAL,
        FOREIGN KEY(order_id) REFERENCES orders(order_id),
        FOREIGN KEY(product_id) REFERENCES products(product_id)
    );
    """))

In [25]:
with ENGINE.begin() as conn:
    conn.execute(text("""
    INSERT OR IGNORE INTO customers(customer_id, country)
    SELECT DISTINCT CustomerID, Country FROM staging;
    """))
    conn.execute(text("""
    INSERT OR IGNORE INTO products(stock_code, description)
    SELECT DISTINCT StockCode, Description FROM staging;
    """))        
    conn.execute(text("DELETE FROM orders;"))
    conn.execute(text("""
    INSERT INTO orders(invoice_no, invoice_date, customer_id)
    SELECT DISTINCT InvoiceNo, InvoiceDate, CustomerID FROM staging;
    """))
    conn.execute(text("DELETE FROM order_items;"))
    conn.execute(text("""
    INSERT INTO order_items(order_id, product_id, quantity, unit_price)
    SELECT o.rowid AS order_id, p.product_id, s.Quantity, s.UnitPrice
    FROM staging s
    JOIN orders o ON o.invoice_no = s.InvoiceNo
    JOIN products p ON p.stock_code = s.StockCode;
    """))

In [27]:
run_sql("""
    SELECT * FROM customers LIMIT 5""")

,customer_id,country
0,12346,United Kingdom
1,12347,Iceland
2,12348,Finland
3,12349,Italy
4,12350,Norway


In [28]:
run_sql("""
SELECT * FROM products LIMIT 5""")

,product_id,stock_code,description
0,1,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,2,71053,WHITE METAL LANTERN
2,3,84406B,CREAM CUPID HEARTS COAT HANGER
3,4,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,5,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [29]:
run_sql("""
SELECT * FROM orders LIMIT 5""")

,order_id,invoice_no,invoice_date,customer_id
0,1,536365,2010-12-01 08:26:00.000000,17850
1,2,536366,2010-12-01 08:28:00.000000,17850
2,3,536367,2010-12-01 08:34:00.000000,13047
3,4,536368,2010-12-01 08:34:00.000000,13047
4,5,536369,2010-12-01 08:35:00.000000,13047


In [31]:
run_sql("""
SELECT * FROM order_items LIMIT 5 """)

,order_item_id,order_id,product_id,quantity,unit_price
0,1,1,7,6,4.25
1,2,1,6,2,7.65
2,3,1,2,6,3.39
3,4,1,5,6,3.39
4,5,1,4,6,3.39


In [49]:
etl_py = r"""
import sys
from pathlib import Path
import pandas as pd
from db import ENGINE
from sqlalchemy import text

def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(subset=['CustomerID','InvoiceNo','StockCode','InvoiceDate'])
    df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce')
    df = df[df['InvoiceDate'].notna()]
    df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(0).astype(int)
    df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce').fillna(0.0)
    df = df[df['Quantity'] > 0]
    df['CustomerID'] = df['CustomerID'].astype(int)
    df['StockCode'] = df['StockCode'].astype(str)
    df['Description'] = df['Description'].fillna('').str.strip()
    return df

def build_and_load(df: pd.DataFrame):
    with ENGINE.begin() as conn:
        conn.execute(text('DROP TABLE IF EXISTS staging;'))
    df.to_sql('staging', con=ENGINE, if_exists='replace', index=False)

    with ENGINE.begin() as conn:
        conn.execute(text('''
        CREATE TABLE IF NOT EXISTS customers (
            customer_id INTEGER PRIMARY KEY, 
            country TEXT
        );
    '''))
        conn.execute(text('''
        CREATE TABLE IF NOT EXISTS products (
            product_id INTEGER PRIMARY KEY AUTOINCREMENT,
            stock_code TEXT UNIQUE,
            description TEXT
        );
        '''))
        conn.execute(text('''
        CREATE TABLE IF NOT EXISTS orders (
            order_id INTEGER PRIMARY KEY,
            invoice_no TEXT,
            invoice_date TIMESTAMP,
            customer_id INTEGER,
            FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
        );
        '''))
        conn.execute(text('''
        CREATE TABLE IF NOT EXISTS order_items (
            order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
            order_id INTEGER,
            product_id INTEGER,
            quantity INTEGER,
            unit_price REAL,
            FOREIGN KEY(order_id) REFERENCES orders(order_id),
            FOREIGN KEY(product_id) REFERENCES products(product_id)
        );
        '''))
        conn.execute(text('''
        INSERT OR IGNORE INTO customers(customer_id, country)
        SELECT DISTINCT CustomerID, Country FROM staging;
        '''))
        conn.execute(text('''
        INSERT OR IGNORE INTO products(stock_code, description)
        SELECT DISTINCT StockCode, Description FROM staging;
        '''))
        conn.execute(text('DELETE FROM orders;'))
        conn.execute(text('''
        INSERT INTO orders(invoice_no, invoice_date, customer_id)
        SELECT DISTINCT InvoiceNo, InvoiceDate, CustomerID FROM staging;
        '''))
        conn.execute(text('''DELETE FROM order_items;'''))
        conn.execute(text('''
        INSERT INTO order_items(order_id, product_id, quantity, unit_price)
        SELECT o.rowid AS order_id, p.product_id, s.Quantity,  s.UnitPrice
        FROM staging s
        JOIN orders o ON o.invoice_no = s.InvoiceNo
        JOIN products p ON p.stock_code = s.StockCode;
        '''))

def main(file_path: str):
    p = Path(file_path)
    if not p.exists():
        raise FileNotFoundError(f"{file_path} not found")
    df = pd.read_excel(p)
    df = clean_df(df)
    build_and_load(df)
    print('ETL complete — DB at:', Path(__file__).parent / 'data'/ 'online_retail.db')

if __name__ == '__main__':
    if len(sys.argv) < 2:
        print('Usage: python etl.py /path/to/Online_Retail.xlsx')
        sys.exit(1)
    main(sys.argv[1])
"""
with open('../etl.py','w') as f:
    f.write(etl_py)
print('wrote etl.py')

wrote etl.py


In [50]:
queries_sql = r"""
-- Monthly revenue
SELECT strftime('%Y-%m', o.invoice_date) AS month,
       SUM(oi.quantity * oi.unit_price) AS revenue,
       COUNT(DISTINCT o.order_id) AS orders,
       COUNT(DISTINCT o.customer_id) AS customers
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY month
ORDER BY month;

-- Top products by revenue
SELECT p.product_id, p.description,
       SUM(oi.quantity * oi.unit_price) AS revenue,
       SUM(oi.quantity) AS total_qty
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_id, p.description
ORDER BY revenue DESC
LIMIT 10;

-- Cohort / retention (month offsets)
WITH first_order AS (
  SELECT customer_id, MIN(strftime('%Y-%m', invoice_date)) AS cohort_month
  FROM orders
  GROUP BY customer_id
),
customer_orders AS (
  SELECT customer_id, strftime('%Y-%m', invoice_date) AS order_month
  FROM orders
)
SELECT f.cohort_month, c.order_month,
       COUNT(DISTINCT c.customer_id) AS active_customers
FROM first_order f
JOIN customer_orders c USING(customer_id)
GROUP BY f.cohort_month, c.order_month
ORDER BY f.cohort_month, c.order_month;

-- RFM
WITH latest AS (SELECT MAX(invoice_date) AS max_date FROM orders),
cust AS (
  SELECT o.customer_id,
         CAST((julianday((SELECT max_date FROM latest)) - julianday(MAX(o.invoice_date))) AS INTEGER) AS recency,
         COUNT(DISTINCT o.order_id) AS frequency,
         SUM(oi.quantity * oi.unit_price) AS monetary
  FROM orders o
  JOIN order_items oi ON o.order_id = oi.order_id
  GROUP BY o.customer_id
)
SELECT customer_id, recency, frequency, monetary
FROM cust
ORDER BY monetary DESC
LIMIT 50;
"""
with open('../queries.sql','w') as f:
    f.write(queries_sql)
print('wrote queries.sql')

wrote queries.sql


### streamlit dashboard

In [54]:
app_py = r"""
import streamlit as st
import pandas as pd
import plotly.express as px
from db import run_sql

from pathlib import Path
import streamlit as st
from etl import main as etl_main

# paths (adjust if your layout differs)
ROOT = Path(__file__).parent
DB_PATH = ROOT / "data" / "online_retail.db"
xlsx_PATH = ROOT / "data" / "Online-Retail.xlsx"  

def ensure_db():
    if not DB_PATH.exists() or DB_PATH.stat().st_size == 0:
        st.info("Building local database — running ETL. This may take a moment...")
        try:
            etl_main(str(xlsx_PATH))
            st.success("ETL complete.")
        except FileNotFoundError as e:
            st.error(f"ETL failed: {e}")
            st.stop()
        except Exception as e:
            st.error(f"ETL error: {e}")
            st.stop()

ensure_db()


st.set_page_config(layout='wide', page_title='Online Retail Analytics')
st.title('Online Retail — SQL & Streamlit')
st.markdown('Dataset: Online Retail (transactions). ETL -> SQLite -> Streamlit.')

# KPIs
kpi_q = '''
SELECT
  COALESCE(SUM(oi.quantity * oi.unit_price),0) AS revenue,
  COUNT(DISTINCT o.order_id) AS orders,
  COUNT(DISTINCT o.customer_id) AS customers,
  AVG(oi.quantity * oi.unit_price) AS avg_order_value
FROM order_items oi
JOIN orders o ON o.order_id = oi.order_id;
'''
kpis = run_sql(kpi_q).iloc[0]
c1, c2, c3, c4 = st.columns(4)
c1.metric('Revenue', f"${kpis.revenue:,.0f}")
c2.metric('Orders', int(kpis.orders))
c3.metric('Customers', int(kpis.customers))
c4.metric('Avg Order Value', f"${kpis.avg_order_value:,.2f}")

st.markdown('---')

# Date range filter
dates = run_sql('SELECT MIN(invoice_date) AS min_date, MAX(invoice_date) AS max_date FROM orders;').iloc[0]
min_date = pd.to_datetime(dates.min_date)
max_date = pd.to_datetime(dates.max_date)
start, end = st.date_input('Date range', value=(min_date.date(), max_date.date()), min_value=min_date.date(), max_value=max_date.date())

# Monthly revenue chart
mrev_q = '''
SELECT strftime('%Y-%m', o.invoice_date) AS month,
       SUM(oi.quantity * oi.unit_price) AS revenue
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE date(o.invoice_date) BETWEEN :start AND :end
GROUP BY month
ORDER BY month;
'''
mrev = run_sql(mrev_q, {"start": start.isoformat(), "end": end.isoformat()})
if not mrev.empty:
    fig = px.line(mrev, x='month', y='revenue', title='Monthly Revenue')
    st.plotly_chart(fig, use_container_width=True)

# Top products
st.markdown('### Top products by revenue')
topn = st.slider('Top N products', 5, 50, 10)
top_q = '''
SELECT p.product_id, p.description,
       SUM(oi.quantity * oi.unit_price) AS revenue,
       SUM(oi.quantity) AS total_qty
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON o.order_id = oi.order_id
WHERE date(o.invoice_date) BETWEEN :start AND :end
GROUP BY p.product_id, p.description
ORDER BY revenue DESC
LIMIT :topn;
'''
top = run_sql(top_q, {"start": start.isoformat(), "end": end.isoformat(), "topn": topn})
st.dataframe(top)

st.markdown('---')

# Cohort heatmap
st.markdown('### Cohort retention heatmap')
cohort_q = '''
WITH first_order AS (
  SELECT customer_id, MIN(strftime('%Y-%m', invoice_date)) AS cohort_month
  FROM orders
  GROUP BY customer_id
),
customer_orders AS (
  SELECT customer_id, strftime('%Y-%m', invoice_date) AS order_month
  FROM orders
)
SELECT f.cohort_month, c.order_month,
       COUNT(DISTINCT c.customer_id) AS active_customers
FROM first_order f
JOIN customer_orders c USING(customer_id)
GROUP BY f.cohort_month, c.order_month
ORDER BY f.cohort_month, c.order_month;
'''
cohort_df = run_sql(cohort_q)
if not cohort_df.empty:
    cohort_df['cohort_month'] = pd.to_datetime(cohort_df['cohort_month'] + '-01')
    cohort_df['order_month'] = pd.to_datetime(cohort_df['order_month'] + '-01')
    cohort_df['month_offset'] = ((cohort_df['order_month'].dt.year - cohort_df['cohort_month'].dt.year) * 12 + (cohort_df['order_month'].dt.month - cohort_df['cohort_month'].dt.month))
    pivot = cohort_df.pivot_table(index='cohort_month', columns='month_offset', values='active_customers', aggfunc='sum').fillna(0)
    cohort_sizes = pivot.iloc[:, 0]
    retention = pivot.divide(cohort_sizes, axis=0).clip(0,1)
    fig = px.imshow(retention, labels=dict(x='Months since cohort', y='Cohort month', color='Retention'), x=retention.columns, y=retention.index.strftime('%Y-%m'))
    st.plotly_chart(fig, use_container_width=True)

st.markdown('---')
st.markdown('### RFM top customers (sample)')
rfm_q = '''
WITH latest AS (SELECT MAX(invoice_date) AS max_date FROM orders),
cust AS (
  SELECT o.customer_id,
         CAST((julianday((SELECT max_date FROM latest)) - julianday(MAX(o.invoice_date))) AS INTEGER) AS recency,
         COUNT(DISTINCT o.order_id) AS frequency,
         SUM(oi.quantity * oi.unit_price) AS monetary
  FROM orders o
  JOIN order_items oi ON o.order_id = oi.order_id
  GROUP BY o.customer_id
)
SELECT customer_id, recency, frequency, monetary
FROM cust
ORDER BY monetary DESC
LIMIT 50;
'''
rfm = run_sql(rfm_q)
st.dataframe(rfm)

st.markdown('---')
st.markdown('GitHub: https://github.com/adrian-pbustamante/Online-retail-sql-streamlit | Live app: add link')
"""
with open('../app.py','w') as f:
    f.write(app_py)
print('wrote app.py')

wrote app.py


In [50]:
#!pip3 install -U streamlit

In [33]:
import streamlit as st
import plotly.express as px
##from db import run_sql

In [48]:
st.set_page_config(layout='wide', page_title='Online Retail Analytics')
st.title('Online Retail - SQL & Streamlit')
st.markdown('Dataset: Online Retail (transactions). ETL -> SQLite -> STreamlit.')

#KPIs
kpi_q = '''
SELECT
    COALESCE(SUM(oi.quantity * oi.unit_price),0) AS revenue,
    COUNT(DISTINCT o.order_id) AS orders,
    COUNT(DISTINCT o.customer_id) AS customers,
    AVG(oi.quantity * oi.unit_price) AS avg_order_value
FROM order_items oi
JOIN orders o
    ON o.order_id = oi.order_id;
'''

kpis = run_sql(kpi_q).iloc[0]
c1, c2, c3, c4 = st.columns(4)
c1.metric('Revenue', f"${kpis.revenue:,.0f}")
c2.metric('Orders', int(kpis.orders))
c3.metric('Customers', int(kpis.customers))
c4.metric('Avg Order Value', f"${kpis.avg_order_value:,.2f}")

st.markdown('---')

# Date range filter
dates = run_sql('''
SELECT 
    MIN(invoice_date) AS min_date, 
    MAX(invoice_date) AS max_date
FROM orders;''')
min_date = pd.to_datetime(dates.min_date)
max_date = pd.to_datetime(dates.max_date)
start, end = st.date_input(
    'Date range', 
    value=(min_date[0].date(), max_date[0].date()),
    min_value=min_date[0].date(),
    max_value=max_date[0].date())

#### Monthly revenue chart
mrev_q = '''
SELECT
    strftime('%Y-%m', o.invoice_date) AS month,
    SUM(oi.quantity * oi.unit_price) AS revenue
FROM orders o
JOIN order_items oi 
    ON o.order_id = oi.order_id
WHERE date(o.invoice_date) BETWEEN :start AND :end
GROUP BY month
ORDER BY month;
'''
mrev = run_sql(mrev_q, {'start':start.isoformat(), 'end': end.isoformat()})
if not mrev.empty:
    fig = px.line(mrev, x='month', y='revenue', title='Monthly Revenue')
    st.plotly_chart(fig, use_container_width=True)

#Top products
st.markdown('### Top products by revenue')
topn = st.slider('Top N products', 5, 50, 10)
top_q = '''
SELECT 
    p.product_id,
    p.description,
    SUM(oi.quantity * oi.unit_price) AS revenue,
    SUM(oi.quantity) AS total_qty
FROM order_items oi
JOIN products p
    ON oi.product_id = p.product_id
JOIN orders o
    ON o.order_id = oi.order_id
WHERE date(o.invoice_date) BETWEEN :start AND :end
GROUP BY p.product_id, p.description
ORDER BY revenue DESC
LIMIT :topn;
'''
top = run_sql(top_q, {'start': start.isoformat(), 'end': end.isoformat(), 'topn':topn})
st.dataframe(top)

st.markdown('----')

#Cohort heatmap

st.markdown('### Cohort retention heatmap')
cohort_q = '''
WITH first_order AS(
    SELECT 
        customer_id, 
        MIN(strftime('%Y-%m', invoice_date)) AS cohort_month
    FROM orders
    GROUP BY customer_id
),
customer_orders AS (
    SELECT
        customer_id, 
        strftime('%Y-%m', invoice_date) AS order_month
    FROM orders
)

SELECT
    f.cohort_month,
    c.order_month,
    COUNT(DISTINCT c.customer_id) AS active_customers
FROM first_order f
JOIN customer_orders c USING(customer_id)
GROUP BY f.cohort_month, c.order_month
ORDER BY f.cohort_month, c.order_month
'''

cohort_df = run_sql(cohort_q)
if not cohort_df.empty:
    cohort_df['cohort_month'] = pd.to_datetime(cohort_df['cohort_month'] + '-01')
    cohort_df['order_month'] = pd.to_datetime(cohort_df['order_month'] + '-01')
    cohort_df['month_offset'] = ((cohort_df['order_month'].dt.year - cohort_df['cohort_month'].dt.year) * 12 
                                 + (cohort_df['order_month'].dt.month - cohort_df['cohort_month'].dt.month))
    pivot = cohort_df.pivot_table(index='cohort_month', columns='month_offset', values='active_customers', aggfunc='sum').fillna(0)
    cohort_sizes = pivot.iloc[:,0]
    retention = pivot.divide(cohort_sizes, axis=0).clip(0,1)
    fig = px.imshow(retention, labels=dict(x='Months since cohort', y='Cohort month', color='Retention'), x=retention.columns, y=retention.index.strftime('%Y-%m'))
    st.plotly_chart(fig, use_container_width=True)

st.markdown('---')
st.markdown('### RFM top customers (sample)')
rfm_q = '''
WITH latest AS (
    SELECT
        MAX(invoice_date) AS max_date 
    FROM orders
),
cust AS (
    SELECT 
        o.customer_id,
        CAST((julianday((SELECT max_date FROM latest)) - julianday(MAX(o.invoice_date))) AS INTEGER) AS recency,
        COUNT(DISTINCT o.order_id) AS frequency,
        SUM(oi.quantity * oi.unit_price) AS monetary
    FROM orders o
    JOIN order_items oi
        ON o.order_id = oi.order_id
    GROUP BY o.customer_id
)
SELECT 
    customer_id,
    recency,
    frequency,
    monetary
FROM cust
ORDER BY monetary DESC
LIMIT 50;
'''

rfm = run_sql(rfm_q)
st.dataframe(rfm)

st.markdown('----')
st.markdown('GitHub: add linnk | Live app: add link')        

2025-09-14 15:03:29.500 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 15:03:29.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 15:03:29.503 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 15:03:29.504 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 15:03:29.506 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 15:03:29.507 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 15:03:29.508 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-14 15:03:29.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [41]:
max_date[0].date()

datetime.date(2011, 12, 9)